![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

### Project Requirements:
 - Columns containing categories with only two factors must be stored as Booleans (bool).
 - Columns containing integers only must be stored as 32-bit integers (int32).
 - Columns containing floats must be stored as 16-bit floats (float16).
 - Columns containing nominal categorical data must be stored as the category data type.
 - Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
 - The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [64]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [84]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Print the data types of each column
print(ds_jobs_transformed.dtypes)



student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                float64
dtype: object


In [ ]:
# Check which columns hava null values
print(ds_jobs_transformed.isnull().sum())

student_id                   0
city                         0
city_development_index       0
gender                    4508
relevant_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
job_change                   0
dtype: int64


##### Columns containing categories with only two factors must be stored as Booleans (bool).

In [86]:
# Determine which categorical columns have two values
print(ds_jobs_transformed.nunique())

student_id                19158
city                        123
city_development_index       93
gender                        3
relevant_experience           2
enrolled_university           3
education_level               5
major_discipline              6
experience                   22
company_size                  8
company_type                  6
last_new_job                  6
training_hours              241
job_change                    2
dtype: int64


In [87]:
# Check the values of the columns with only two factors
print(ds_jobs_transformed["relevant_experience"].value_counts())
print(ds_jobs_transformed["job_change"].value_counts())

relevant_experience
Has relevant experience    13792
No relevant experience      5366
Name: count, dtype: int64
job_change
0.0    14381
1.0     4777
Name: count, dtype: int64


In [88]:
# Create a dictionary for the old value to new value pairs
updated_relevant_experience = {"Has relevant experience": True, "No relevant experience": False}
updated_job_change= {0.0: False, 1.0: True}

# Update the current values of the relevant_experience column
ds_jobs_transformed["relevant_experience"] = ds_jobs_transformed["relevant_experience"].replace(updated_relevant_experience)

# Update the current values of the job_change column
ds_jobs_transformed["job_change"] = ds_jobs_transformed["job_change"].replace(updated_job_change)

# Check if the expected changes were done correctly
print(ds_jobs_transformed["relevant_experience"].value_counts())
print(ds_jobs_transformed["job_change"].value_counts())

# Check if the column data type is now bool
print(ds_jobs_transformed[["relevant_experience","job_change"]].dtypes)

relevant_experience
True     13792
False     5366
Name: count, dtype: int64
job_change
False    14381
True      4777
Name: count, dtype: int64
relevant_experience    bool
job_change             bool
dtype: object


C:\Users\Kendot\AppData\Local\Temp\ipykernel_10212\652304986.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ds_jobs_transformed["relevant_experience"] = ds_jobs_transformed["relevant_experience"].replace(updated_relevant_experience)
C:\Users\Kendot\AppData\Local\Temp\ipykernel_10212\652304986.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ds_jobs_transformed["job_change"] = ds_jobs_transformed["job_change"].replace(updated_job_change)


##### Columns containing integers only must be stored as 32-bit integers (int32) and Columns containing floats must be stored as 16-bit floats (float16).

In [ ]:
# Change the datatype of student_id and training hours as int32
ds_jobs_transformed["student_id"] = ds_jobs_transformed["student_id"].astype("int32")
ds_jobs_transformed["training_hours"] = ds_jobs_transformed["training_hours"].astype("int32")

# Change the datatype of city_development_index as float16
ds_jobs_transformed["city_development_index"] = ds_jobs_transformed["city_development_index"].astype("float16")

# Verify if the changes were done
print(ds_jobs_transformed[["student_id","training_hours","city_development_index"]].dtypes)


student_id                  int32
training_hours              int32
city_development_index    float16
dtype: object


##### Columns containing nominal categorical data must be stored as the category data type

In [ ]:
# Fill missing values
ds_jobs_transformed["gender"] = ds_jobs_transformed["gender"].fillna("Prefer not to say")

# Check if there are no null values 
print(ds_jobs_transformed["gender"].value_counts())

# Change the datatype to category
ds_jobs_transformed["gender"] = ds_jobs_transformed["gender"].astype("category")

# Check the datatype
print(ds_jobs_transformed["gender"].dtype)


gender
Male                 1654
Prefer not to say     380
Female                145
Other                  22
Name: count, dtype: int64
category


In [ ]:
print(ds_jobs_transformed["city"].value_counts(dropna=False))

# Change the datatype to category
ds_jobs_transformed["city"] = ds_jobs_transformed["city"].astype("category")

# Check the datatype
print(ds_jobs_transformed["city"].dtype)

city
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_111       3
city_129       3
city_121       3
city_140       1
city_171       1
Name: count, Length: 123, dtype: int64
category


In [ ]:
# Fill missing values
ds_jobs_transformed["major_discipline"] = ds_jobs_transformed["major_discipline"].fillna("No information")

# Check if there are no null values
print(ds_jobs_transformed["major_discipline"].value_counts(dropna=False))

# Change the datatype to category
ds_jobs_transformed["major_discipline"] = ds_jobs_transformed["major_discipline"].astype("category")

# Check the datatype
print(ds_jobs_transformed["major_discipline"].dtype)


major_discipline
STEM               14492
No information      2813
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: count, dtype: int64
category


In [ ]:
# Fill missing values
ds_jobs_transformed["company_type"] = ds_jobs_transformed["company_type"].fillna("No information")

# Check if there are no null values
print(ds_jobs_transformed["company_type"].value_counts(dropna=False))

# Change the datatype to category
ds_jobs_transformed["company_type"] = ds_jobs_transformed["company_type"].astype("category")

# Check the datatype

print(ds_jobs_transformed["company_type"].dtype)

company_type
Pvt Ltd                9817
No information         6140
Funded Startup         1001
Public Sector           955
Early Stage Startup     603
NGO                     521
Other                   121
Name: count, dtype: int64
category


##### Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column

In [ ]:
# Fill missing values
ds_jobs_transformed["enrolled_university"] = ds_jobs_transformed["enrolled_university"].fillna("No information")

# Change the datatype to category
ds_jobs_transformed["enrolled_university"] = ds_jobs_transformed["enrolled_university"].astype("category")

# Re-order the categories
ds_jobs_transformed["enrolled_university"] = ds_jobs_transformed["enrolled_university"].cat.reorder_categories(
    new_categories=["no_enrollment","Part time course","Full time course","No information"],
    ordered=True
)

# Verify the ordered categories
print(ds_jobs_transformed["enrolled_university"].value_counts(sort=False))

# Check the datatype
print(ds_jobs_transformed["enrolled_university"].dtype)




enrolled_university
no_enrollment       1960
Part time course     103
Full time course     122
No information        16
Name: count, dtype: int64
category


In [111]:
# Fill missing values
ds_jobs_transformed["education_level"] = ds_jobs_transformed["education_level"].fillna("No information")

# Change the datatype to category
ds_jobs_transformed["education_level"] = ds_jobs_transformed["education_level"].astype("category")

# Re-order the categories
ds_jobs_transformed["education_level"] = ds_jobs_transformed["education_level"].cat.reorder_categories(
    new_categories=["Primary School","High School","Graduate","Masters","Phd","No information"],
    ordered=True,
)
# Verify the ordered categories
print(ds_jobs_transformed["education_level"].value_counts(sort=False))

# Check the datatype
print(ds_jobs_transformed["education_level"].dtype)

education_level
Primary School       7
High School         75
Graduate          1243
Masters            744
Phd                115
No information      17
Name: count, dtype: int64
category


In [112]:
# Fill missing values
ds_jobs_transformed["company_size"] = ds_jobs_transformed["company_size"].fillna("No information")

# Change the datatype to category
ds_jobs_transformed["company_size"] = ds_jobs_transformed["company_size"].astype("category")

# Re-order the categories
ds_jobs_transformed["company_size"] = ds_jobs_transformed["company_size"].cat.reorder_categories(
    new_categories=["<10","10-49","50-99","100-499","500-999","1000-4999","5000-9999","10000+","No information"],
    ordered=True
)
# Verify the ordered categories
print(ds_jobs_transformed["company_size"].value_counts(sort=False))

# Check the datatype
print(ds_jobs_transformed["company_size"].dtype)

company_size
<10                  0
10-49                0
50-99                0
100-499              0
500-999              0
1000-4999          796
5000-9999          310
10000+            1095
No information       0
Name: count, dtype: int64
category


In [ ]:
# Temporarily replace the <20 to an integer
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].str.replace(">20","21",regex=False)

# Temporarily replace the <1 to an integer
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].str.replace("<1","0",regex=False)

# Fill null values with an integer
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].fillna("-1")

# Change the datatype to int
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].astype(int)

# Create a list for the experience groups
experience_groups = ["<1","1-5","6-9","10-15","16-20",">20","No information"]

# Create a list of conditions for the experience groups
conditions = [
    (ds_jobs_transformed["experience"] == 0),
    ((ds_jobs_transformed["experience"] >= 1) & (ds_jobs_transformed["experience"] <= 5)),
    ((ds_jobs_transformed["experience"] >= 6) & (ds_jobs_transformed["experience"] <= 9)),
    ((ds_jobs_transformed["experience"] >= 10) & (ds_jobs_transformed["experience"] <= 15)),
    ((ds_jobs_transformed["experience"] >= 16) & (ds_jobs_transformed["experience"] <= 20)),
     (ds_jobs_transformed["experience"] > 20),
     (ds_jobs_transformed["experience"] == -1)
]
# Change the values in the experience column based on the conditions
ds_jobs_transformed["experience"] = np.select(conditions,experience_groups,default="No information")

# Change the datatyoe to category
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].astype("category")

# Re-order the categories
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].cat.reorder_categories(
    new_categories=["<1","1-5","6-9","10-15","16-20",">20","No information"],
    ordered=True
)
# Verify the ordered categories
print(ds_jobs_transformed["experience"].value_counts(sort=False))



experience
<1                 522
1-5               5863
6-9               4026
10-15             3814
16-20             1582
>20               3286
No information      65
Name: count, dtype: int64


In [ ]:
# Fill missing values
ds_jobs_transformed["last_new_job"] = ds_jobs_transformed["last_new_job"].fillna("No information")

# Change datatype to category
ds_jobs_transformed["last_new_job"] = ds_jobs_transformed["last_new_job"].astype("category")

# Re-order the categories
ds_jobs_transformed["last_new_job"] = ds_jobs_transformed["last_new_job"].cat.reorder_categories(
    new_categories=["never","1","2","3","4",">4","No information"],
    ordered=True
)
# Check the ordered categories
print(ds_jobs_transformed["last_new_job"].value_counts(sort=False))

last_new_job
never             2452
1                 8040
2                 2900
3                 1024
4                 1029
>4                3290
No information     423
Name: count, dtype: int64


##### The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies

In [ ]:
# Filter the data based on the conditions provided
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed["experience"] >= "10-15") &
    (ds_jobs_transformed["experience"] != "No information") &
    (ds_jobs_transformed["company_size"] >= "1000-4999") &
    (ds_jobs_transformed["company_size"] != "No information")
]

# Print the first few rows
ds_jobs_transformed.head()

c:\Users\Kendot\Documents\Code\Data Camp Projects\DataCamp-Projects\venv\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,Prefer not to say,True,no_enrollment,Graduate,STEM,16-20,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,16-20,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,10-15,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16-20,1000-4999,Public Sector,2,14,False


In [ ]:
# Check the memory usage of the original dataset
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [ ]:
# Check the memory usage of the transformed dataset
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8747 entries, 0 to 19155
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              8747 non-null   int32   
 1   city                    8747 non-null   category
 2   city_development_index  8747 non-null   float16 
 3   gender                  8747 non-null   category
 4   relevant_experience     8747 non-null   bool    
 5   enrolled_university     8747 non-null   category
 6   education_level         8747 non-null   category
 7   major_discipline        8747 non-null   category
 8   experience              8747 non-null   category
 9   company_size            8747 non-null   category
 10  company_type            8747 non-null   category
 11  last_new_job            8747 non-null   category
 12  training_hours          8747 non-null   int32   
 13  job_change              8747 non-null   bool    
dtypes: bool(2), category(9), flo